# Loading and preprocessing files

In [2]:
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as layers
import numpy as np
import random

SEED = 24
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

def preprocess_file(f_name):
  df = pd.read_csv(f_name, sep=';', index_col=0).reset_index(drop=True)
  df[['S', 'H', 'D', 'C']] = df.PBN.str.split('.', expand=True)
  colors = ['S', 'H', 'D', 'C']
  values = ['A', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2']
  for color in colors:
    for val in values:
      df.loc[:, val+color] = 0.
      df.loc[df[color].str.find(val)!=-1, val+color] = 1.
  df.loc[:, 'Pts'] = (df['AS']+df['AH']+df['AD']+df['AC'])*4+(df['KS']+df['KH']+df['KD']+df['KC'])*3+(df['QS']+df['QH']+df['QD']+df['QC'])*2+(df['JS']+df['JH']+df['JD']+df['JC'])
  df['#S'] = df.loc[:, 'AS':'2S'].sum(axis=1)
  df['#H'] = df.loc[:, 'AH':'2H'].sum(axis=1)
  df['#D'] = df.loc[:, 'AD':'2D'].sum(axis=1)
  df['#C'] = df.loc[:, 'AC':'2C'].sum(axis=1)
  df['#A'] = df['AS']+df['AH']+df['AD']+df['AC']
  df['#K'] = df['KS']+df['KH']+df['KD']+df['KC']
  df['#Q'] = df['QS']+df['QH']+df['QD']+df['QC']
  df['#J'] = df['JS']+df['JH']+df['JD']+df['JC']

  df.loc[:, ['l0', 'l1', 'l2', 'l3', 'l4', 'cp', 'cc', 'cd', 'ch', 'cs', 'cn']] = 0.
  df.loc[df['OPENING']=='PASS', 'OPENING'] = '0P'
  df.loc[df['OPENING'].str[0]=='0', 'l0'] = 1.
  df.loc[df['OPENING'].str[0]=='1', 'l1'] = 1.
  df.loc[df['OPENING'].str[0]=='2', 'l2'] = 1.
  df.loc[df['OPENING'].str[0]=='3', 'l3'] = 1.
  df.loc[df['OPENING'].str[0]=='4', 'l4'] = 1.
  df.loc[df['OPENING'].str[1]=='P', 'cp'] = 1.
  df.loc[df['OPENING'].str[1]=='C', 'cc'] = 1.
  df.loc[df['OPENING'].str[1]=='D', 'cd'] = 1.
  df.loc[df['OPENING'].str[1]=='H', 'ch'] = 1.
  df.loc[df['OPENING'].str[1]=='S', 'cs'] = 1.
  df.loc[df['OPENING'].str[1]=='N', 'cn'] = 1.
  df.drop(['PBN', 'OPENING', 'S', 'H', 'D', 'C'], axis=1, inplace=True)
  return df


df_train = preprocess_file("../data/openings/train.csv")
train_x, val_x = df_train.loc[:699, 'AS':'#J'], df_train.loc[700:, 'AS':'#J']
train_y, val_y = df_train.loc[:699, 'l0':'cn'], df_train.loc[700:, 'l0':'cn']

df_test = preprocess_file("../data/openings/test.csv")
test_x, test_y = df_test.loc[:, 'AS':'#J'], df_test.loc[:, 'l0':'cn']

2023-04-25 10:31:32.031207: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-25 10:31:32.632795: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-25 10:31:32.635198: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 10:31:34.584545: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Base model

In [3]:
class SaveBestModel(tf.keras.callbacks.Callback):
  def __init__(self, save_best_metric='val_loss', this_max=False):
    self.save_best_metric = save_best_metric
    self.max = this_max
    if this_max:
      self.best = float('-inf')
    else:
      self.best = float('inf')

  def on_epoch_end(self, epoch, logs=None):
    metric_value = logs[self.save_best_metric]
    if self.max:
      if metric_value > self.best:
        self.best = metric_value
        self.best_weights = self.model.get_weights()
    else:
      if metric_value < self.best:
        self.best = metric_value
        self.best_weights= self.model.get_weights()


class BridgeModel():
  def __init__(self):
    self.model_level = None
    self.model_color = None

  def preprocess_input(self, x):
    pass

  def preprocess_output(self, y):
    return y.loc[:, 'l0':'l4'], y.loc[:, 'cp':'cn']

  def train(self, x, y, val_x, val_y, epochs=50, batch_size=10, verbose=False):
    x = self.preprocess_input(x)
    y_l, y_c = self.preprocess_output(y)
    val_x = self.preprocess_input(val_x)
    val_y_l, val_y_c = self.preprocess_output(val_y)
    save_best_model = SaveBestModel(save_best_metric="val_accuracy", this_max=True)
    hist_l = self.model_level.fit(x, y_l, epochs=epochs, batch_size=batch_size, validation_data=(val_x, val_y_l), verbose=verbose, callbacks=[save_best_model])
    self.model_level.set_weights(save_best_model.best_weights)
    save_best_model = SaveBestModel(save_best_metric="val_accuracy", this_max=True)
    hist_c = self.model_color.fit(x, y_c, epochs=epochs, batch_size=batch_size, validation_data=(val_x, val_y_c), verbose=verbose, callbacks=[save_best_model])
    self.model_color.set_weights(save_best_model.best_weights)
    return hist_l, hist_c

  def predict(self, data):
    levels = self.model_level.predict(self.preprocess_input(data))
    colors = self.model_color.predict(self.preprocess_input(data))
    out_df = pd.DataFrame(np.concatenate([levels, colors], axis=1), columns=['l0', 'l1', 'l2', 'l3', 'l4', 'cp', 'cc', 'cd', 'ch', 'cs', 'cn'], index=data.index)
    out_df.loc[out_df.loc[:, 'l0':'l4'].idxmax(axis=1)=='l0', 'cp'] = 1
    out_df.loc[out_df.loc[:, 'l0':'l4'].idxmax(axis=1)=='l0', 'cc':'cn'] = 0
    out_df.loc[out_df.loc[:, 'cp':'cn'].idxmax(axis=1)=='cp', 'l0'] = 1
    out_df.loc[out_df.loc[:, 'cp':'cn'].idxmax(axis=1)=='cp', 'l1':'l4'] = 0
    return out_df

  def evaluate(self, x, y):
    preds = self.predict(x)
    good_levels = preds.loc[:, 'l0':'l4'].idxmax(axis=1)==y.loc[:, 'l0':'l4'].idxmax(axis=1)
    good_colors = preds.loc[:, 'cp':'cn'].idxmax(axis=1)==y.loc[:, 'cp':'cn'].idxmax(axis=1)
    return (good_levels & good_colors).sum()/len(x)
      
  def load_weights(self, level_fname, color_fname):
    self.model_level.load_weights(level_fname)
    self.model_color.load_weights(color_fname)

  def save_weights(self, level_fname, color_fname):
    self.model_level.save_weights(level_fname)
    self.model_color.save_weights(color_fname)

# Raw model

In [4]:
class ModelRaw(BridgeModel):
  def __init__(self):
    super().__init__()
    self.model_level = tf.keras.models.Sequential([
        layers.Input(shape=(52)),
        layers.Dense(8, activation='relu'),
        layers.Dense(16, activation='relu'),
        layers.Dense(8, activation='relu'),
        layers.Dense(5, activation='softmax')
    ])

    self.model_color = tf.keras.models.Sequential([
        layers.Input(shape=(52)),
        layers.Dense(8, activation='relu'),
        layers.Dense(16, activation='relu'),
        layers.Dense(8, activation='relu'),
        layers.Dense(6, activation='softmax')
    ])
    self.model_level.compile(optimizer = tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics = ["accuracy"])
    self.model_color.compile(optimizer = tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics = ["accuracy"])

  def preprocess_input(self, x):
    return x.loc[:, 'AS':'2C']

In [5]:
model_raw = ModelRaw()
model_raw.train(train_x, train_y, val_x, val_y)
model_raw.train(train_x, train_y, val_x, val_y)
print(model_raw.evaluate(train_x, train_y))
print(model_raw.evaluate(val_x, val_y))
print(model_raw.evaluate(test_x, test_y))

22/22 [==============================] - 0s 2ms/step
0.82
10/10 [==============================] - 0s 1ms/step
0.6566666666666666
7/7 [==============================] - 0s 2ms/step
0.6948356807511737


# Heuristic model

In [6]:
class ModelHeuristic(BridgeModel):
  def __init__(self):
    super().__init__()
    self.model_level = tf.keras.models.Sequential([
        layers.Input(shape=9),
        layers.Dense(8, activation='relu'),
        layers.Dense(16, activation='relu'),
        layers.Dense(8, activation='relu'),
        layers.Dense(5, activation='softmax')
    ])

    self.model_color = tf.keras.models.Sequential([
        layers.Input(shape=(9)),
        layers.Dense(8, activation='relu'),
        layers.Dense(16, activation='relu'),
        layers.Dense(8, activation='relu'),
        layers.Dense(6, activation='softmax')
    ])
    self.model_level.compile(optimizer = tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics = ["accuracy"])
    self.model_color.compile(optimizer = tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics = ["accuracy"])

  def preprocess_input(self, x):
    return x.loc[:, 'Pts':'#J']

In [7]:
model_heuristic = ModelHeuristic()
model_heuristic.train(train_x, train_y, val_x, val_y)
model_heuristic.train(train_x, train_y, val_x, val_y)
print(model_heuristic.evaluate(train_x, train_y))
print(model_heuristic.evaluate(val_x, val_y))
print(model_heuristic.evaluate(test_x, test_y))

22/22 [==============================] - 0s 1ms/step
0.8342857142857143
10/10 [==============================] - 0s 1ms/step
0.8066666666666666
7/7 [==============================] - 0s 1ms/step
0.7887323943661971


# Combined model

In [8]:
class ModelCombined(BridgeModel):
  def __init__(self):
    super().__init__()
    self.model_level = tf.keras.models.Sequential([
        layers.Input(shape=61),
        layers.Dense(8, activation='relu'),
        layers.Dense(16, activation='relu'),
        layers.Dense(8, activation='relu'),
        layers.Dense(5, activation='softmax')
    ])

    self.model_color = tf.keras.models.Sequential([
        layers.Input(shape=(61)),
        layers.Dense(8, activation='relu'),
        layers.Dense(16, activation='relu'),
        layers.Dense(8, activation='relu'),
        layers.Dense(6, activation='softmax')
    ])
    self.model_level.compile(optimizer = tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics = ["accuracy"])
    self.model_color.compile(optimizer = tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics = ["accuracy"])

  def preprocess_input(self, x):
    return x

In [9]:
model_combined = ModelCombined()
model_combined.train(train_x, train_y, val_x, val_y)
model_combined.train(train_x, train_y, val_x, val_y)
print(model_combined.evaluate(train_x, train_y))
print(model_combined.evaluate(val_x, val_y))
print(model_combined.evaluate(test_x, test_y))

22/22 [==============================] - 0s 1ms/step
0.7914285714285715
10/10 [==============================] - 0s 2ms/step
0.7033333333333334
7/7 [==============================] - 0s 2ms/step
0.7183098591549296


# Expert model

In [10]:
class ModelExpert(BridgeModel):
  def __init__(self):
    super().__init__()

  def preprocess_input(self, x):
    return x

  def predict_one(self, row):
    out = pd.Series(index=['l0', 'l1', 'l2', 'l3', 'l4', 'cp', 'cc', 'cd', 'ch', 'cs', 'cn'], dtype=float)
    out[:] = 0
    if row['Pts'] >= 18:
      out['l1'] = 1
      out['cc'] = 1
    elif row['Pts'] >= 12:
      if row['#S'] >= 5:
        out['l1'] = 1
        out['cs'] = 1
      elif row['#H'] >= 5:
        out['l1'] = 1
        out['ch'] = 1
      elif row['Pts'] >= 15:
        if ((row['#S':'#C'].min()==2 and row['#S':'#C'].max()==4) or 
            (row['#C']==5 and row['#S':'#D'].min()==2 and row['#S':'#D'].max()==3) or
            (row['#D']==5 and row[['#S', '#H', '#C']].min()==2 and row[['#S', '#H', '#C']].max()==3)):
          out['l1'] = 1
          out['cn'] = 1
        elif row['#D']>=row['#C']:
          out['l1'] = 1
          out['cd'] = 1
        else:
          out['l1'] = 1
          out['cc'] = 1
      elif (row['#D']>=5 or
            (row['#D']==4 and row['#C']==5) or
            (row['#D']==4 and row[['#S', '#H', '#C']].min()==1)):
        out['l1'] = 1
        out['cd'] = 1
      elif (row['#C']>=6 or
            ((row['#H']==4 or row['#S']==4) and row['#C']==5)):
            out['l2'] = 1
            out['cc'] = 1
      else:
        out['l1'] = 1
        out['cc'] = 1
    elif row['Pts'] >= 7:
      if row['#S']==6 or row['#H']==6:
        out['l2'] = 1
        out['cd'] = 1
      elif row['#H']==5 and row[['#S', '#D', '#C']].max()>=5:
        out['l2'] = 1
        out['ch'] = 1
      elif row['#S']==5 and row['#D':'#C'].max()>=5:
        out['l2'] = 1
        out['cs'] = 1
      elif row['#D']>=5 and row['#C']>=5:
        out['l2'] = 1
        out['cn'] = 1
      elif row['#C']==7:
        out['l3'] = 1
        out['cc'] = 1
      elif row['#D']==7:
        out['l3'] = 1
        out['cd'] = 1
      elif row['#H']==7:
        out['l3'] = 1
        out['ch'] = 1
      elif row['#S']==7:
        out['l3'] = 1
        out['cs'] = 1
      elif row['#C']==8:
        out['l4'] = 1
        out['cc'] = 1
      elif row['#D']==8:
        out['l4'] = 1
        out['cd'] = 1
      elif row['#H']==8:
        out['l4'] = 1
        out['ch'] = 1
      elif row['#S']==8:
        out['l4'] = 1
        out['cs'] = 1
      else:
        out['l0'] = 1
        out['cp'] = 1
    else:
      out['l0'] = 1
      out['cp'] = 1
    return out

  def predict(self, data):
    return data.apply(self.predict_one, axis=1)

In [11]:
model_expert = ModelExpert()
print(model_expert.evaluate(train_x, train_y))
print(model_expert.evaluate(val_x, val_y))
print(model_expert.evaluate(test_x, test_y))

0.8471428571428572
0.87
0.812206572769953
